In [1]:
def numbirred(n,p): #The number of irred. polys of degree n in F_p[x].
    L = []
    for i in range(1,n+1):
        if ZZ(i).divides(ZZ(n)):
            L.append(p^i*moebius(n/i))
        else:
            pass
    return 1/n*sum(L)

In [2]:
var('i,index')

(i, index)

In [3]:
def ordGSp(g,n): #This is the order of GSp_{2g}(n).
    primeorders = []
    h = factor(n)
    for i in range(0,len(h)):
        l = h[i][0]
        e = h[i][1]
        primeorders.append((l-1)*l^(g^2)*product(l^(2*index)-1,index,1,g)*(l^(2*g^2+g+1))^(e-1)) #I've used product to try to make things easy to change as I increase my dimension g.
    return product(primeorders)

In [4]:
ordGSp(2,5)

37440000

In [5]:
GSpOrders = [0,0]
for n in range(2,1001):
    GSpOrders.append(ordGSp(3,ZZ(n)))

In [6]:
def Frobmatrix3(a5,a4,a3,p): #This is for the minimal order Z[pi,v].
    frob = Matrix([[0,0,0,-p*a4,p,0],[1,0,0,-a3,0,0],[0,1,0,-a4,0,0],[0,0,1,-a5,0,0],[0,0,0,-p*a5,0,p],[0,0,0,-p,0,0]])
    return frob

In [7]:
testfrob = Frobmatrix3(-4,9,-15,2);testfrob

[  0   0   0 -18   2   0]
[  1   0   0  15   0   0]
[  0   1   0  -9   0   0]
[  0   0   1   4   0   0]
[  0   0   0   8   0   2]
[  0   0   0  -2   0   0]

In [8]:
def WeilPolys(p):
    list = []
    R.<x> = QQ[]
    for a5 in range(ceil(-6*sqrt(p)),ceil(6*sqrt(p))): #This is from the first condition.
        for a4 in range(ceil(4*sqrt(p)*abs(a5)-9*p),floor(a5^2/3+3*p)+1): #This is from the second condition.
            for a3 in range(floor(-2*p*a5-2*sqrt(p)*a4-2*p*sqrt(p))+1,ceil(-2*p*a5+2*sqrt(p)*a4+2*p*sqrt(p))): #This is from the fourth condition.
                if -2*a5^3/27+a5*a4/3+p*a5-2/27*(a5^2-3*a4+9*p)^(3/2)<=a3 and a3<= -2*a5^3/27+a5*a4/3+p*a5+2/27*(a5^2-3*a4+9*p)^(3/2): #This is from the third condition.
                    f= R(x^6+a5*x^5+a4*x^4+a3*x^3+p*a4*x^2+p^2*a5*x+p^3)
                    if f.is_irreducible():
                        #list.append(f)
                        list.append([a5,a4,a3])
    return list

In [9]:
Wp2 = WeilPolys(2);len(Wp2)

80

In [10]:
Wp3 = WeilPolys(3); len(Wp3)

348

In [11]:
Wp5 = WeilPolys(5); len(Wp5)

2032

In [12]:
def Mono3(bound,a5,a4,a3,p):
    #bound is the arbitrary upper bound.
    print(a5,a4,a3)
    nonmono = [] #This is to get the list of non-monogenic stuff easily.
    Frob = Frobmatrix3(a5,a4,a3,p)
    #print Frob
    print("****************")
    #print latex(Frob)
    if ZZ(a3).valuation(p) == 0:
        prank = 3
    elif ZZ(a4).valuation(p) == 0:
        prank = 2
    elif ZZ(a5).valuation(p) == 0:
        prank = 1
    else:
        prank = 0
    print("p-rank is %i." % prank)
    for n in range(2,bound):
        try:
            if gcd(Frob.determinant(),n)!=1:
                #print "n is %i and the determinant is not 1." % n
                pass
            else:
                GSpsize = GSpOrders[n]
                #print "GSpsize is %i." % GSpsize
                R = Integers(ZZ(n)) #This works better than the quotient ring I was using.
                V = GL(2*3,R) #I will need to change this for different g.
                F = V(Frob)
                Fj = F
                j=1
                while GSpsize/numbirred(j,p)>j:
                    if not Fj.is_one():
                        #print "j is %i." % j
                        Fj = Fj*F
                        j = j+1
                    else:
                        ord = j
                        print("***********************************")
                        print("n is %i." % n)
                        print("The order of the Frobenius is %i." % ord)
                        print("We have %i polys of the proper degree in Z/%iZ[x]." % (numbirred(ord,p),p))
                        split = GSpsize/ord
                        print("The number of primes %i splits into has to be %s." % (p,split))
                        nonmono.append(str(n))
                        break
                pass
        except:
            pass
    print(nonmono)
    return "%i & %i & %i & %i & %s \\\\ \\hline" %(a5,a4,a3,prank,", ".join(nonmono)) #join concatenates and the quotes tell it what to put between the elts of the list.

In [13]:
Mono3(50,-2,2,-2,2)

-2 2 -2
****************
p-rank is 0.
***********************************
n is 3.
The order of the Frobenius is 20.
We have 52377 polys of the proper degree in Z/2Z[x].
The number of primes 2 splits into has to be 917070336.
***********************************
n is 9.
The order of the Frobenius is 60.
We have 19215358392200893 polys of the proper degree in Z/2Z[x].
The number of primes 2 splits into has to be 9592879626553886208.
['3', '9']


'-2 & 2 & -2 & 0 & 3, 9 \\\\ \\hline'

In [12]:
Mono3(50,-4, 9, -15,2)

-4 9 -15
****************
p-rank is 3.
***********************************
n is 7.
The order of the Frobenius is 42.
We have 104715342801 polys of the proper degree in Z/2Z[x].
The number of primes 2 splits into has to be 78130633887129600.
***********************************
n is 11.
The order of the Frobenius is 70.
We have 16865594581186450683 polys of the proper degree in Z/2Z[x].
The number of primes 2 splits into has to be 1048369348798130304000.
***********************************
n is 23.
The order of the Frobenius is 77.
We have 1962541914958813595274 polys of the proper degree in Z/2Z[x].
The number of primes 2 splits into has to be 11256236457209520538360135680.
***********************************
n is 29.
The order of the Frobenius is 28.
We have 9586395 polys of the proper degree in Z/2Z[x].
The number of primes 2 splits into has to be 5126732189998128935888697216000.
***********************************
n is 43.
The order of the Frobenius is 42.
We have 104715342801 polys 

'-4 & 9 & -15 & 3 & 7, 11, 23, 29, 43 \\\\ \\hline'

In [13]:
TheList2=[]
for i in range(0, len(Wp2)):
    TheList2.append(Mono3(200,Wp2[i][0],Wp2[i][1],Wp2[i][2],2))

-4 9 -15
****************
p-rank is 3.
***********************************
n is 7.
The order of the Frobenius is 42.
We have 104715342801 polys of the proper degree in Z/2Z[x].
The number of primes 2 splits into has to be 78130633887129600.
***********************************
n is 11.
The order of the Frobenius is 70.
We have 16865594581186450683 polys of the proper degree in Z/2Z[x].
The number of primes 2 splits into has to be 1048369348798130304000.
***********************************
n is 23.
The order of the Frobenius is 77.
We have 1962541914958813595274 polys of the proper degree in Z/2Z[x].
The number of primes 2 splits into has to be 11256236457209520538360135680.
***********************************
n is 29.
The order of the Frobenius is 28.
We have 9586395 polys of the proper degree in Z/2Z[x].
The number of primes 2 splits into has to be 5126732189998128935888697216000.
***********************************
n is 43.
The order of the Frobenius is 42.
We have 104715342801 polys 

In [14]:
print("\n".join(TheList2))

-4 & 9 & -15 & 3 & 7, 11, 23, 29, 43, 71, 87, 113, 127 \\ \hline
-3 & 2 & 1 & 3 & 7, 11, 29, 43, 71, 87, 113, 127 \\ \hline
-3 & 5 & -7 & 3 &  \\ \hline
-3 & 6 & -9 & 3 & 3, 9, 27, 153 \\ \hline
-2 & 0 & 3 & 3 & 107, 149 \\ \hline
-2 & 1 & 0 & 2 & 3, 5, 11, 55, 83 \\ \hline
-2 & 1 & 1 & 3 & 3 \\ \hline
-2 & 2 & -2 & 0 & 3, 9 \\ \hline
-2 & 2 & -1 & 3 & 3 \\ \hline
-2 & 3 & -6 & 2 & 73 \\ \hline
-2 & 3 & -5 & 3 & 3, 9, 27, 59, 63 \\ \hline
-2 & 3 & -3 & 3 & 5, 83, 131 \\ \hline
-2 & 4 & -6 & 0 & 3 \\ \hline
-2 & 5 & -7 & 3 & 3, 7 \\ \hline
-1 & -1 & 2 & 2 & 3, 9 \\ \hline
-1 & -1 & 4 & 2 & 3, 7 \\ \hline
-1 & -1 & 5 & 3 & 3, 9 \\ \hline
-1 & 0 & -1 & 3 & 3 \\ \hline
-1 & 0 & 0 & 1 &  \\ \hline
-1 & 0 & 1 & 3 & 5, 15 \\ \hline
-1 & 0 & 3 & 3 & 7, 77, 103 \\ \hline
-1 & 1 & -3 & 3 & 3 \\ \hline
-1 & 1 & -1 & 3 & 3 \\ \hline
-1 & 1 & 0 & 2 & 3, 7 \\ \hline
-1 & 2 & -5 & 3 & 3, 5 \\ \hline
-1 & 2 & -3 & 3 & 3 \\ \hline
-1 & 2 & -2 & 1 & 3 \\ \hline
-1 & 2 & -1 & 3 & 3, 5, 7, 9, 35, 75 \\ \h

In [13]:
Wp3

[[-5, 13, -25],
 [-5, 15, -31],
 [-4, 6, -7],
 [-4, 7, -10],
 [-4, 8, -13],
 [-4, 9, -17],
 [-4, 10, -21],
 [-4, 10, -20],
 [-4, 10, -19],
 [-4, 11, -22],
 [-4, 12, -23],
 [-3, 0, 8],
 [-3, 1, 5],
 [-3, 2, 1],
 [-3, 2, 2],
 [-3, 3, -1],
 [-3, 4, -6],
 [-3, 4, -5],
 [-3, 5, -9],
 [-3, 5, -8],
 [-3, 5, -7],
 [-3, 5, -5],
 [-3, 6, -12],
 [-3, 6, -11],
 [-3, 6, -10],
 [-3, 7, -15],
 [-3, 7, -13],
 [-3, 7, -11],
 [-3, 8, -17],
 [-3, 8, -16],
 [-3, 8, -14],
 [-3, 8, -13],
 [-3, 9, -17],
 [-3, 9, -15],
 [-2, -2, 9],
 [-2, -2, 11],
 [-2, -1, 6],
 [-2, -1, 7],
 [-2, 0, 2],
 [-2, 0, 3],
 [-2, 0, 4],
 [-2, 0, 5],
 [-2, 0, 7],
 [-2, 1, -1],
 [-2, 1, 0],
 [-2, 1, 1],
 [-2, 1, 2],
 [-2, 2, -5],
 [-2, 2, -3],
 [-2, 2, -2],
 [-2, 2, -1],
 [-2, 2, 1],
 [-2, 3, -7],
 [-2, 3, -6],
 [-2, 3, -4],
 [-2, 3, -2],
 [-2, 3, -1],
 [-2, 4, -11],
 [-2, 4, -10],
 [-2, 4, -9],
 [-2, 4, -8],
 [-2, 4, -7],
 [-2, 4, -5],
 [-2, 4, -4],
 [-2, 4, -3],
 [-2, 5, -11],
 [-2, 5, -10],
 [-2, 5, -8],
 [-2, 5, -6],
 [-2, 5, -5],

In [14]:
AbbrevList3 = [[0, -2, -3],
 [0, -2, -2],
 [0, -2, -1],
 [0, -2, 1],
 [0, -2, 2],
 [0, -2, 3],
 [0, -1, -6],
 [0, -1, -5],
 [0, -1, -4],
 [0, -1, -2],
 [0, -1, -1],
 [0, -1, 1],
 [0, -1, 2],
 [0, -1, 4],
 [0, -1, 5],
 [0, -1, 6],
 [0, 0, -9],
 [0, 0, -7],
 [0, 0, -6],
 [0, 0, -5],
 [0, 0, -4],
 [0, 0, -3],
 [0, 0, -2],
 [0, 0, -1],
 [0, 0, 1],
 [0, 0, 2],
 [0, 0, 3],
 [0, 0, 4],
 [0, 0, 5],
 [0, 0, 6],
 [0, 0, 7],
 [0, 0, 9],
 [0, 1, -6],
 [0, 1, -5],
 [0, 1, -4],
 [0, 1, -2],
 [0, 1, -1],
 [0, 1, 1],
 [0, 1, 2],
 [0, 1, 4],
 [0, 1, 5],
 [0, 1, 6],
 [0, 2, -7],
 [0, 2, -5],
 [0, 2, -4],
 [0, 2, -3],
 [0, 2, -2],
 [0, 2, -1],
 [0, 2, 1],
 [0, 2, 2],
 [0, 2, 3],
 [0, 2, 4],
 [0, 2, 5],
 [0, 2, 7],
 [0, 3, -3],
 [0, 3, -2],
 [0, 3, -1],
 [0, 3, 1],
 [0, 3, 2],
 [0, 3, 3],
 [0, 4, -3],
 [0, 4, -1],
 [0, 4, 1],
 [0, 4, 3],
 [0, 5, -2],
 [0, 5, -1],
 [0, 5, 1],
 [0, 5, 2],
 [0, 6, -1],
 [0, 6, 1],]

In [16]:
List3=[]
for i in range(0, len(AbbrevList3)):
    List3.append(Mono3(200,AbbrevList3[i][0],AbbrevList3[i][1],AbbrevList3[i][2],3))

0 -2 -3
****************
p-rank is 2.
***********************************
n is 2.
The order of the Frobenius is 9.
We have 2184 polys of the proper degree in Z/3Z[x].
The number of primes 3 splits into has to be 161280.
***********************************
n is 4.
The order of the Frobenius is 18.
We have 21522228 polys of the proper degree in Z/3Z[x].
The number of primes 3 splits into has to be 338228674560.
***********************************
n is 8.
The order of the Frobenius is 36.
We have 4169295414140220 polys of the proper degree in Z/3Z[x].
The number of primes 3 splits into has to be 709316941310853120.
['2', '4', '8']
0 -2 -2
****************
p-rank is 3.
***********************************
n is 2.
The order of the Frobenius is 6.
We have 116 polys of the proper degree in Z/3Z[x].
The number of primes 3 splits into has to be 241920.
***********************************
n is 4.
The order of the Frobenius is 12.
We have 44220 polys of the proper degree in Z/3Z[x].
The number of 

In [18]:
print("\n".join(List3))

0 & -2 & -3 & 2 & 2, 4, 8 \\ \hline
0 & -2 & -2 & 3 & 2, 4, 8, 16 \\ \hline
0 & -2 & -1 & 3 & 2, 4, 8 \\ \hline
0 & -2 & 1 & 3 & 2, 4, 8 \\ \hline
0 & -2 & 2 & 3 & 2, 4, 8, 16 \\ \hline
0 & -2 & 3 & 2 & 2, 4, 8 \\ \hline
0 & -1 & -6 & 2 & 2, 4, 8 \\ \hline
0 & -1 & -5 & 3 &  \\ \hline
0 & -1 & -4 & 3 & 2, 4, 8, 16, 32 \\ \hline
0 & -1 & -2 & 3 & 2, 4, 8 \\ \hline
0 & -1 & -1 & 3 &  \\ \hline
0 & -1 & 1 & 3 &  \\ \hline
0 & -1 & 2 & 3 & 2, 4, 8 \\ \hline
0 & -1 & 4 & 3 & 2, 4, 8, 16, 32 \\ \hline
0 & -1 & 5 & 3 &  \\ \hline
0 & -1 & 6 & 2 & 2, 4, 8 \\ \hline
0 & 0 & -9 & 0 & 2, 4, 7, 8, 13, 14, 19, 26, 28, 37, 38, 40, 52, 56, 65, 70, 73, 74, 76, 80, 91, 95, 104, 112, 122, 124, 130, 133, 140, 146, 148, 152, 182, 185, 190 \\ \hline
0 & 0 & -7 & 3 & 2, 4, 5, 7, 8, 10, 11, 13, 14, 17, 19, 20, 22, 25, 26, 28, 35, 38, 40, 49, 56, 70, 76, 80, 85, 91, 112, 119, 133, 140, 145, 154, 175 \\ \hline
0 & 0 & -6 & 0 & 2, 4, 8, 11, 13, 14, 16, 17, 19, 22, 26, 28, 34, 38, 40, 43, 44, 56, 68, 86, 112, 11

In [12]:
LongList3=[]
for i in range(0, len(Wp3)):
    LongList3.append(Mono3(200,Wp3[i][0],Wp3[i][1],Wp3[i][2],3))

-5 13 -25
****************
p-rank is 3.
***********************************
n is 2.
The order of the Frobenius is 7.
We have 312 polys of the proper degree in Z/3Z[x].
The number of primes 3 splits into has to be 207360.
***********************************
n is 4.
The order of the Frobenius is 14.
We have 341484 polys of the proper degree in Z/3Z[x].
The number of primes 3 splits into has to be 434865438720.
***********************************
n is 5.
The order of the Frobenius is 8.
We have 810 polys of the proper degree in Z/3Z[x].
The number of primes 3 splits into has to be 228501000000000.
***********************************
n is 8.
The order of the Frobenius is 28.
We have 817028131140 polys of the proper degree in Z/3Z[x].
The number of primes 3 splits into has to be 911978924542525440.
***********************************
n is 20.
The order of the Frobenius is 56.
We have 9345493446916743936090180 polys of the proper degree in Z/3Z[x].
The number of primes 3 splits into has to b

In [13]:
LongList3

['-5 & 13 & -25 & 3 & 2, 4, 5, 8, 20, 25, 40, 80 \\\\ \\hline',
 '-5 & 15 & -31 & 3 & 2, 4, 8, 14, 28, 91, 104, 182 \\\\ \\hline',
 '-4 & 6 & -7 & 3 & 2, 4, 5, 8, 40, 80 \\\\ \\hline',
 '-4 & 7 & -10 & 3 & 2, 4, 8 \\\\ \\hline',
 '-4 & 8 & -13 & 3 & 2, 4, 8 \\\\ \\hline',
 '-4 & 9 & -17 & 3 &  \\\\ \\hline',
 '-4 & 10 & -21 & 2 & 2, 4, 7, 8, 56, 79, 112 \\\\ \\hline',
 '-4 & 10 & -20 & 3 & 2, 4, 8, 16, 32 \\\\ \\hline',
 '-4 & 10 & -19 & 3 & 2, 4, 8 \\\\ \\hline',
 '-4 & 11 & -22 & 3 & 2, 4, 5, 8, 10, 13, 20 \\\\ \\hline',
 '-4 & 12 & -23 & 3 & 2, 4, 8, 13 \\\\ \\hline',
 '-3 & 0 & 8 & 3 & 2, 4, 8 \\\\ \\hline',
 '-3 & 1 & 5 & 3 & 2, 4, 8, 77 \\\\ \\hline',
 '-3 & 2 & 1 & 3 & 2, 4, 5 \\\\ \\hline',
 '-3 & 2 & 2 & 3 & 2, 4, 8, 16 \\\\ \\hline',
 '-3 & 3 & -1 & 3 & 2, 4, 8, 17 \\\\ \\hline',
 '-3 & 4 & -6 & 2 & 2, 4, 8, 16 \\\\ \\hline',
 '-3 & 4 & -5 & 3 & 2, 4, 79 \\\\ \\hline',
 '-3 & 5 & -9 & 2 & 2, 4, 8, 16, 32 \\\\ \\hline',
 '-3 & 5 & -8 & 3 & 2, 4 \\\\ \\hline',
 '-3 & 5 & -7 & 3

In [14]:
print("\n".join(LongList3))

-5 & 13 & -25 & 3 & 2, 4, 5, 8, 20, 25, 40, 80 \\ \hline
-5 & 15 & -31 & 3 & 2, 4, 8, 14, 28, 91, 104, 182 \\ \hline
-4 & 6 & -7 & 3 & 2, 4, 5, 8, 40, 80 \\ \hline
-4 & 7 & -10 & 3 & 2, 4, 8 \\ \hline
-4 & 8 & -13 & 3 & 2, 4, 8 \\ \hline
-4 & 9 & -17 & 3 &  \\ \hline
-4 & 10 & -21 & 2 & 2, 4, 7, 8, 56, 79, 112 \\ \hline
-4 & 10 & -20 & 3 & 2, 4, 8, 16, 32 \\ \hline
-4 & 10 & -19 & 3 & 2, 4, 8 \\ \hline
-4 & 11 & -22 & 3 & 2, 4, 5, 8, 10, 13, 20 \\ \hline
-4 & 12 & -23 & 3 & 2, 4, 8, 13 \\ \hline
-3 & 0 & 8 & 3 & 2, 4, 8 \\ \hline
-3 & 1 & 5 & 3 & 2, 4, 8, 77 \\ \hline
-3 & 2 & 1 & 3 & 2, 4, 5 \\ \hline
-3 & 2 & 2 & 3 & 2, 4, 8, 16 \\ \hline
-3 & 3 & -1 & 3 & 2, 4, 8, 17 \\ \hline
-3 & 4 & -6 & 2 & 2, 4, 8, 16 \\ \hline
-3 & 4 & -5 & 3 & 2, 4, 79 \\ \hline
-3 & 5 & -9 & 2 & 2, 4, 8, 16, 32 \\ \hline
-3 & 5 & -8 & 3 & 2, 4 \\ \hline
-3 & 5 & -7 & 3 & 2, 4, 8 \\ \hline
-3 & 5 & -5 & 3 & 2, 4, 8, 16, 32, 83, 104 \\ \hline
-3 & 6 & -12 & 0 & 2, 4, 8 \\ \hline
-3 & 6 & -11 & 3 & 2, 4, 22 \\ 

In [13]:
Wp5

[[-8, 32, -85],
 [-8, 34, -93],
 [-7, 23, -55],
 [-7, 24, -59],
 [-7, 25, -63],
 [-7, 26, -68],
 [-7, 27, -73],
 [-7, 27, -72],
 [-7, 27, -71],
 [-7, 28, -75],
 [-7, 29, -77],
 [-6, 15, -29],
 [-6, 16, -33],
 [-6, 17, -38],
 [-6, 17, -37],
 [-6, 18, -41],
 [-6, 19, -47],
 [-6, 19, -46],
 [-6, 20, -51],
 [-6, 20, -50],
 [-6, 20, -49],
 [-6, 20, -47],
 [-6, 21, -55],
 [-6, 21, -54],
 [-6, 21, -53],
 [-6, 22, -59],
 [-6, 22, -57],
 [-6, 22, -55],
 [-6, 23, -62],
 [-6, 23, -61],
 [-6, 23, -59],
 [-6, 23, -58],
 [-6, 24, -63],
 [-6, 24, -61],
 [-5, 6, 1],
 [-5, 7, -3],
 [-5, 8, -8],
 [-5, 8, -7],
 [-5, 9, -12],
 [-5, 9, -11],
 [-5, 10, -16],
 [-5, 10, -15],
 [-5, 11, -21],
 [-5, 11, -19],
 [-5, 12, -26],
 [-5, 12, -25],
 [-5, 12, -24],
 [-5, 13, -30],
 [-5, 13, -29],
 [-5, 13, -28],
 [-5, 13, -27],
 [-5, 13, -25],
 [-5, 14, -34],
 [-5, 14, -33],
 [-5, 14, -32],
 [-5, 14, -31],
 [-5, 15, -39],
 [-5, 15, -37],
 [-5, 15, -36],
 [-5, 15, -35],
 [-5, 15, -33],
 [-5, 16, -42],
 [-5, 16, -41],
 [-

In [14]:
AbbrevList5 = [[0, -4, -4],
 [0, -4, -3],
 [0, -4, -2],
 [0, -4, -1],
 [0, -4, 1],
 [0, -4, 2],
 [0, -4, 3],
 [0, -4, 4],
 [0, -3, -7],
 [0, -3, -6],
 [0, -3, -5],
 [0, -3, -4],
 [0, -3, -3],
 [0, -3, -2],
 [0, -3, -1],
 [0, -3, 1],
 [0, -3, 2],
 [0, -3, 3],
 [0, -3, 4],
 [0, -3, 5],
 [0, -3, 6],
 [0, -3, 7],
 [0, -2, -13],
 [0, -2, -12],
 [0, -2, -11],
 [0, -2, -10],
 [0, -2, -9],
 [0, -2, -8],
 [0, -2, -7],
 [0, -2, -6],
 [0, -2, -5],
 [0, -2, -3],
 [0, -2, -2],
 [0, -2, -1],
 [0, -2, 1],
 [0, -2, 2],
 [0, -2, 3],
 [0, -2, 5],
 [0, -2, 6],
 [0, -2, 7],
 [0, -2, 8],
 [0, -2, 9],
 [0, -2, 10],
 [0, -2, 11],
 [0, -2, 12],
 [0, -2, 13],
 [0, -1, -17],
 [0, -1, -16],
 [0, -1, -14],
 [0, -1, -13],
 [0, -1, -12],
 [0, -1, -11],
 [0, -1, -10],
 [0, -1, -9],
 [0, -1, -8],
 [0, -1, -7],
 [0, -1, -6],
 [0, -1, -5],
 [0, -1, -4],
 [0, -1, -3],
 [0, -1, -2],
 [0, -1, -1],
 [0, -1, 1],
 [0, -1, 2],
 [0, -1, 3],
 [0, -1, 4],
 [0, -1, 5],
 [0, -1, 6],
 [0, -1, 7],
 [0, -1, 8],
 [0, -1, 9],
 [0, -1, 10],
 [0, -1, 11],
 [0, -1, 12],
 [0, -1, 13],
 [0, -1, 14],
 [0, -1, 16],
 [0, -1, 17],
 [0, 0, -21],
 [0, 0, -20],
 [0, 0, -19],
 [0, 0, -17],
 [0, 0, -16],
 [0, 0, -15],
 [0, 0, -13],
 [0, 0, -12],
 [0, 0, -11],
 [0, 0, -10],
 [0, 0, -9],
 [0, 0, -8],
 [0, 0, -7],
 [0, 0, -6],
 [0, 0, -5],
 [0, 0, -3],
 [0, 0, -2],
 [0, 0, -1],
 [0, 0, 1],
 [0, 0, 2],
 [0, 0, 3],
 [0, 0, 5],
 [0, 0, 6],
 [0, 0, 7],
 [0, 0, 8],
 [0, 0, 9],
 [0, 0, 10],
 [0, 0, 11],
 [0, 0, 12],
 [0, 0, 13],
 [0, 0, 15],
 [0, 0, 16],
 [0, 0, 17],
 [0, 0, 19],
 [0, 0, 20],
 [0, 0, 21],
 [0, 1, -19],
 [0, 1, -18],
 [0, 1, -17],
 [0, 1, -16],
 [0, 1, -14],
 [0, 1, -12],
 [0, 1, -11],
 [0, 1, -10],
 [0, 1, -9],
 [0, 1, -7],
 [0, 1, -6],
 [0, 1, -5],
 [0, 1, -4],
 [0, 1, -3],
 [0, 1, -2],
 [0, 1, -1],
 [0, 1, 1],
 [0, 1, 2],
 [0, 1, 3],
 [0, 1, 4],
 [0, 1, 5],
 [0, 1, 6],
 [0, 1, 7],
 [0, 1, 9],
 [0, 1, 10],
 [0, 1, 11],
 [0, 1, 12],
 [0, 1, 14],
 [0, 1, 16],
 [0, 1, 17],
 [0, 1, 18],
 [0, 1, 19],
 [0, 2, -17],
 [0, 2, -16],
 [0, 2, -15],
 [0, 2, -14],
 [0, 2, -13],
 [0, 2, -11],
 [0, 2, -10],
 [0, 2, -9],
 [0, 2, -8],
 [0, 2, -7],
 [0, 2, -6],
 [0, 2, -5],
 [0, 2, -4],
 [0, 2, -3],
 [0, 2, -2],
 [0, 2, -1],
 [0, 2, 1],
 [0, 2, 2],
 [0, 2, 3],
 [0, 2, 4],
 [0, 2, 5],
 [0, 2, 6],
 [0, 2, 7],
 [0, 2, 8],
 [0, 2, 9],
 [0, 2, 10],
 [0, 2, 11],
 [0, 2, 13],
 [0, 2, 14],
 [0, 2, 15],
 [0, 2, 16],
 [0, 2, 17],
 [0, 3, -15],
 [0, 3, -14],
 [0, 3, -13],
 [0, 3, -12],
 [0, 3, -10],
 [0, 3, -8],
 [0, 3, -7],
 [0, 3, -6],
 [0, 3, -5],
 [0, 3, -4],
 [0, 3, -3],
 [0, 3, -2],
 [0, 3, -1],
 [0, 3, 1],
 [0, 3, 2],
 [0, 3, 3],
 [0, 3, 4],
 [0, 3, 5],
 [0, 3, 6],
 [0, 3, 7],
 [0, 3, 8],
 [0, 3, 10],
 [0, 3, 12],
 [0, 3, 13],
 [0, 3, 14],
 [0, 3, 15],
 [0, 4, -13],
 [0, 4, -12],
 [0, 4, -11],
 [0, 4, -9],
 [0, 4, -8],
 [0, 4, -7],
 [0, 4, -5],
 [0, 4, -4],
 [0, 4, -3],
 [0, 4, -2],
 [0, 4, -1],
 [0, 4, 1],
 [0, 4, 2],
 [0, 4, 3],
 [0, 4, 4],
 [0, 4, 5],
 [0, 4, 7],
 [0, 4, 8],
 [0, 4, 9],
 [0, 4, 11],
 [0, 4, 12],
 [0, 4, 13],
 [0, 5, -11],
 [0, 5, -10],
 [0, 5, -8],
 [0, 5, -7],
 [0, 5, -6],
 [0, 5, -5],
 [0, 5, -4],
 [0, 5, -2],
 [0, 5, -1],
 [0, 5, 1],
 [0, 5, 2],
 [0, 5, 4],
 [0, 5, 5],
 [0, 5, 6],
 [0, 5, 7],
 [0, 5, 8],
 [0, 5, 10],
 [0, 5, 11],
 [0, 6, -9],
 [0, 6, -7],
 [0, 6, -6],
 [0, 6, -5],
 [0, 6, -4],
 [0, 6, -3],
 [0, 6, -2],
 [0, 6, -1],
 [0, 6, 1],
 [0, 6, 2],
 [0, 6, 3],
 [0, 6, 4],
 [0, 6, 5],
 [0, 6, 6],
 [0, 6, 7],
 [0, 6, 9],
 [0, 7, -6],
 [0, 7, -5],
 [0, 7, -4],
 [0, 7, -2],
 [0, 7, -1],
 [0, 7, 1],
 [0, 7, 2],
 [0, 7, 4],
 [0, 7, 5],
 [0, 7, 6],
 [0, 8, -7],
 [0, 8, -5],
 [0, 8, -4],
 [0, 8, -3],
 [0, 8, -2],
 [0, 8, -1],
 [0, 8, 1],
 [0, 8, 2],
 [0, 8, 3],
 [0, 8, 4],
 [0, 8, 5],
 [0, 8, 7],
 [0, 9, -3],
 [0, 9, -2],
 [0, 9, -1],
 [0, 9, 1],
 [0, 9, 2],
 [0, 9, 3],
 [0, 10, -3],
 [0, 10, -1],
 [0, 10, 1],
 [0, 10, 3],
 [0, 11, -2],
 [0, 11, -1],
 [0, 11, 1],
 [0, 11, 2],
 [0, 12, -1],
 [0, 12, 1]]

In [15]:
TheList5=[]
for i in range(0, len(AbbrevList5)):
    TheList5.append(Mono3(200,AbbrevList5[i][0],AbbrevList5[i][1],AbbrevList5[i][2],5))

0 -4 -4
****************
p-rank is 3.
***********************************
n is 2.
The order of the Frobenius is 6.
We have 2580 polys of the proper degree in Z/5Z[x].
The number of primes 5 splits into has to be 241920.
***********************************
n is 4.
The order of the Frobenius is 12.
We have 20343700 polys of the proper degree in Z/5Z[x].
The number of primes 5 splits into has to be 507343011840.
***********************************
n is 8.
The order of the Frobenius is 12.
We have 20343700 polys of the proper degree in Z/5Z[x].
The number of primes 5 splits into has to be 2127950823932559360.
***********************************
n is 16.
The order of the Frobenius is 24.
We have 2483526855452500 polys of the proper degree in Z/5Z[x].
The number of primes 5 splits into has to be 4462636326311814726942720.
['2', '4', '8', '16']
0 -4 -3
****************
p-rank is 3.
***********************************
n is 3.
The order of the Frobenius is 8.
We have 48750 polys of the proper d

In [16]:
print("\n".join(TheList5))

0 & -4 & -4 & 3 & 2, 4, 8, 16 \\ \hline
0 & -4 & -3 & 3 & 3, 4, 8, 9, 16 \\ \hline
0 & -4 & -2 & 3 & 2, 4, 8 \\ \hline
0 & -4 & -1 & 3 & 4, 8, 16 \\ \hline
0 & -4 & 1 & 3 & 4, 8, 16 \\ \hline
0 & -4 & 2 & 3 & 2, 4, 8 \\ \hline
0 & -4 & 3 & 3 & 3, 4, 8, 9, 16 \\ \hline
0 & -4 & 4 & 3 & 2, 4, 8, 16 \\ \hline
0 & -3 & -7 & 3 &  \\ \hline
0 & -3 & -6 & 3 & 2, 3, 4, 9, 42, 84 \\ \hline
0 & -3 & -5 & 2 &  \\ \hline
0 & -3 & -4 & 3 & 2, 4, 8, 12, 16 \\ \hline
0 & -3 & -3 & 3 & 3, 9, 12, 24 \\ \hline
0 & -3 & -2 & 3 & 2, 4 \\ \hline
0 & -3 & -1 & 3 &  \\ \hline
0 & -3 & 1 & 3 &  \\ \hline
0 & -3 & 2 & 3 & 2, 4 \\ \hline
0 & -3 & 3 & 3 & 3, 9, 12, 24 \\ \hline
0 & -3 & 4 & 3 & 2, 4, 8, 12, 16 \\ \hline
0 & -3 & 5 & 2 &  \\ \hline
0 & -3 & 6 & 3 & 2, 3, 4, 9, 42, 84 \\ \hline
0 & -3 & 7 & 3 &  \\ \hline
0 & -2 & -13 & 3 & 4, 13 \\ \hline
0 & -2 & -12 & 3 & 2, 3, 4, 6, 8, 12, 16, 18, 24, 36, 48, 72, 96 \\ \hline
0 & -2 & -11 & 3 & 4, 11, 33 \\ \hline
0 & -2 & -10 & 2 & 2, 4, 8 \\ \hline
0 & -2 & 